In [1]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Dropout, merge
from keras.layers import LSTM,Bidirectional,GRU,SimpleRNN
import pandas as pd
from keras.layers import Conv2D, MaxPooling2D, Input

import tensorflow as tf
from keras.layers import Conv1D, GlobalMaxPooling1D,GlobalAveragePooling1D,MaxPooling1D, AveragePooling1D
from scipy.stats import pearsonr
from sklearn.cross_validation import train_test_split

Using Theano backend.


In [2]:
import keras.backend as K

def pearson(X,Y):
    #return K.mean(K.abs(y_true-y_pred))
    if K.sum(X)==0:return 0
    n = K.sum(X)/K.mean(X)
    if n==0:return 0
    num = K.sum(X*Y) - K.sum(X)*K.sum(Y)/n
    den = (K.sum(X*X) - K.square(K.sum(X))/n)*(K.sum(Y*Y) - K.square(K.sum(Y))/n)
    den = K.sqrt(den)
    if den==0 : return 0
    return num/den

def pearson1(X,Y):
    #return K.mean(K.abs(y_true-y_pred))
    if K.sum(X)==0:return 0
    n = K.sum(X)/K.mean(X)
    if n==0:return 0
    num = K.sum(X*Y) - K.sum(X)*K.sum(Y)/n
    den = (K.sum(X*X) - K.square(K.sum(X))/n)*(K.sum(Y*Y) - K.square(K.sum(Y))/n)
    den = K.sqrt(den)
    if den==0 : return 0
    return -1 * num/den

# prepare train, dev, test data

# ensemble of feed-forward NN and CNN/LSTM for each emotion

## anger

In [32]:
# import pickle
# with open('temp/x_train_1.pickle', 'rb') as handle:
#     x_train_1 = pickle.load(handle)
# with open('temp/x_train_2.pickle', 'rb') as handle:
#     x_train_2 = pickle.load(handle)
# with open('temp/x_test_1.pickle', 'rb') as handle:
#     x_test_1 = pickle.load(handle)
# with open('temp/x_test_2.pickle', 'rb') as handle:
#     x_test_2 = pickle.load(handle)
# with open('temp/y_train.pickle', 'rb') as handle:
#     y_train = pickle.load(handle)
# with open('temp/y_test.pickle', 'rb') as handle:
#     y_test = pickle.load(handle)
    
lex_train = np.load('../intermediates/train/anger/lexicon.npy')
lex_val = np.load('../intermediates/val/anger/lexicon.npy')
lex_test = np.load('../intermediates/test/anger/lexicon.npy')
lex_train = np.concatenate([lex_train, lex_val], axis=0)
print lex_train.shape, lex_test.shape

(941, 43) (760, 43)


In [62]:
ypred = np.zeros(760)
for i in range(6) : 
    input1 = Input(shape=(443,))
    l1 = Dense(256, init='normal', activation='relu')(input1)
    l1 = Dropout(p=0.2)(l1)
    l1 = Dense(128, init='normal', activation='relu')(l1)
    #l1 = Dense(50, init='normal', activation='relu')(l1)
    #l1 = Dropout(p=0.2)(l1)
    #l1 = Dense(25, init='normal', activation='relu')(l1)
    #o1 = Dense(1, init='normal')(l1)

    #cnn-lstm-1
    input2 = Input(shape=(50,400,))
    l2 = Conv1D(128,3,activation='relu')(input2)
    #l2 = Conv1D(64,3,activation='relu')(l2)
    l2 = GlobalMaxPooling1D()(l2)
    l2 = Dropout(p=0.2)(l2)
    l2 = Dense(128,activation='relu')(l2)
    #l2 = Dense(50,activation='relu')(l2)
    #l2 = Dense(25,activation='relu')(l2)
    #o2 = Dense(1)(l2)

    input3 = Input(shape=(43,))
    
    merged_output = merge([l1, l2, input3], mode='concat', concat_axis=-1)
    merged_output = Dense(128, activation='relu')(merged_output)
    merged_output = Dropout(p=0.2)(merged_output)
    merged_output = Dense(64, activation='relu')(merged_output)
    merged_output = Dense(32, activation='relu')(merged_output)
    
    merged_output = Dropout(p=0.2)(merged_output)
    predictions = Dense(1, activation='sigmoid')(merged_output)

    model = Model(input=[input1, input2, input3], output=predictions)
    model.compile(loss='mae', optimizer='adam', metrics=[pearson])
    model.fit([x_train_1['anger'], x_train_2['anger'], lex_train], y_train['anger'], 
              nb_epoch=25, verbose=2, batch_size=8,
             )
    tmp = model.predict([x_test_1['anger'], x_test_2['anger'], lex_test])
    print pearsonr(tmp[:,0], y_test['anger'])
    ypred += tmp[:,0]
    
#     feature_layer = K.function([[model.layers[0].output,model.layers[5].output], K.learning_phase()], [model.layers[2].output])
#     features = feature_layer([x_test_1['anger'], x_test_2['anger'] , 1 ])
#     print features.shape
    
print pearsonr(ypred/7.0, y_test['anger'])
np.save('anger1.npy', ypred/7.0)

Epoch 1/25
4s - loss: 0.1247 - pearson: 0.4250
Epoch 2/25
4s - loss: 0.1145 - pearson: 0.5559
Epoch 3/25
3s - loss: 0.1031 - pearson: 0.6632
Epoch 4/25
3s - loss: 0.0938 - pearson: 0.7324
Epoch 5/25
3s - loss: 0.0841 - pearson: 0.7795
Epoch 6/25
3s - loss: 0.0779 - pearson: 0.8004
Epoch 7/25
4s - loss: 0.0725 - pearson: 0.8369
Epoch 8/25
3s - loss: 0.0700 - pearson: 0.8541
Epoch 9/25
3s - loss: 0.0628 - pearson: 0.8732
Epoch 10/25
4s - loss: 0.0612 - pearson: 0.8672
Epoch 11/25
4s - loss: 0.0607 - pearson: 0.8879
Epoch 12/25
4s - loss: 0.0600 - pearson: 0.9062
Epoch 13/25
4s - loss: 0.0530 - pearson: 0.9136
Epoch 14/25
4s - loss: 0.0524 - pearson: 0.9250
Epoch 15/25
4s - loss: 0.0530 - pearson: 0.9111
Epoch 16/25
5s - loss: 0.0488 - pearson: 0.9227
Epoch 17/25
6s - loss: 0.0495 - pearson: 0.9259
Epoch 18/25
6s - loss: 0.0488 - pearson: 0.9355
Epoch 19/25
6s - loss: 0.0487 - pearson: 0.9267
Epoch 20/25
6s - loss: 0.0473 - pearson: 0.9420
Epoch 21/25
6s - loss: 0.0438 - pearson: 0.9400
E

KeyboardInterrupt: 